In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
271,AFG,Asia,Afghanistan,2020-09-27,39227.0,35.0,26.143,1453.0,0.0,1.714,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
474,ALB,Europe,Albania,2020-09-27,13153.0,108.0,132.429,375.0,2.0,2.429,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
746,DZA,Africa,Algeria,2020-09-27,50914.0,160.0,184.429,1711.0,4.0,6.571,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-26,France,527769.0,0.0,31715.0,0.0,0.0,0.0
0,2020-09-26,China,90460.0,0.0,4744.0,0.0,0.0,0.0
0,2020-09-26,Italy,307770.0,0.0,35838.0,0.0,0.0,0.0
0,2020-09-26,Spain,726666.0,0.0,31312.0,0.0,0.0,0.0
0,2020-09-26,United States,7101480.0,0.0,204878.0,0.0,0.0,0.0
0,2020-09-26,World,33084907.0,0.0,998206.0,0.0,0.0,0.0
0,2020-09-26,United Kingdom,428001.0,0.0,41997.0,0.0,0.0,0.0
0,2020-09-26,Germany,284550.0,0.0,9469.0,0.0,0.0,0.0
0,2020-09-26,Iran,442730.0,0.0,25422.0,0.0,0.0,0.0
0,2020-09-26,Turkey,312773.0,0.0,7918.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-27,France,527769.0,527446.0,31715.0,31700.0,323.0,15.0
0,2020-09-27,China,90460.0,90456.0,4744.0,4739.0,4.0,5.0
0,2020-09-27,Italy,307770.0,308104.0,35838.0,35818.0,-334.0,20.0
0,2020-09-26,Spain,726666.0,716481.0,31312.0,31232.0,10185.0,80.0
0,2020-09-27,United States,7101480.0,7078798.0,204878.0,204497.0,22682.0,381.0
0,2020-09-27,World,33084907.0,32916707.0,998206.0,995147.0,168200.0,3059.0
0,2020-09-27,United Kingdom,428001.0,429277.0,41997.0,41971.0,-1276.0,26.0
0,2020-09-27,Germany,284550.0,284140.0,9469.0,9457.0,410.0,12.0
0,2020-09-27,Iran,442730.0,443086.0,25422.0,25394.0,-356.0,28.0
0,2020-09-27,Turkey,312773.0,312966.0,7918.0,7929.0,-193.0,-11.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")